# Teoria da Computação: Avaliação sobre Indecidibilidade

---

### Nome: Mateus Sousa Araújo

> *Prof. Davi Romero de Vasconcelos, daviromero@ufc.br, Universidade Federal do Ceará, Campus de Quixadá.*



In [ ]:
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    CYELLOW = '\33[33m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

#@title Implementação de MT em Python de Funções para Indecidibilidade (*beta*)
#@markdown Esta célula contém a implementação em Python das Máquinas de Turing 

#@markdown >*Execute esta célula (`ctrl+enter` ou clicando no botão ao lado) para que o ambiente seja carregado com as funções implementadas.*

!pip install xmltodict
import xmltodict
from graphviz import Digraph
from time import sleep
from IPython.display import Image
import ipywidgets as widgets
from IPython.display import clear_output
from google.colab import files
import pandas as pd

S_EPSILON = '\u03B5' # ε
S_BLK = ' '
S_BLK_U = '\u2294' # ⊔
S_BLK_BOX = '\u2610' # ☐
S_LEFT_ANGLE_BRACKET = '\u2329' # 〈
S_RIGHT_ANGLE_BRACKET = '\u232a' # 〉

def uploadArquivos():
  uploaded = files.upload()

def salvarArquivo(filename,input):
  arquivo = open(filename, "w")
  arquivo.write(input)
  arquivo.close()  

def carregaArquivo(filename):
  with open(filename) as fd:
    return fd.read()

class MTNDMF: # Máquina de Turing Não-Determinística Multi-fita

    default = {'blank':' ','right':'R','left':'L','stationary':'S','separador':';','separador_tape':'|'}

    @staticmethod
    def valida(Q={}, Sigma={}, Gamma={}, delta={}, q0=0, blank_as=S_BLK,  F=set()):
      b = True
      ERROR = []
      if not blank_as in Gamma:
        b = False
        ERROR.append(f"{blank_as} não está em Gamma")
      if blank_as in Sigma:
        b = False
        ERROR.append(f"{blank_as} está em Sigma")
      if not q0 in Q:
        b = False
        ERROR.append(f"{q0} não está em Q")
      if not F.issubset(Q):
        b = False
        ERROR.append(f"{F} não é subconjunto de Q")
      if not Sigma.issubset(Gamma):
        b = False
        ERROR.append(f"Sigma não é subconjunto de Gamma")
      ntapes = max([len(t)-1 for t in delta])
      for d in delta.keys():
        q = d[0]
        if not q in Q:
          b = False
          ERROR.append(f"Na transição delta({d}), {q} não está em Q")
        for i in range(1,ntapes+1):
          if not d[i] in Gamma:
            b = False
            ERROR.append(f"Na transição delta({d}), {d[i]} não está em Gamma")
        for t in delta[d]:
          q = t[0]
          if not q in Q:
            b = False
            ERROR.append(f"Na transição delta({d})={t}, {q} não está em Q")
          for i in range(1,ntapes+1):
            if not t[i] in Gamma:
              b = False
              ERROR.append(f"Na transição delta({d})={t}, {t[i]} não está em Gamma")
          for i in range(ntapes+1,2*ntapes+1):
            if not t[i] in {'S','L','R'}:
              b = False
              ERROR.append(f"Na transição delta({d})={t}, {t[i]} não é um movimento válido")
      return b, ERROR

    @staticmethod
    def jffToMT(input_jff):
          """ Convert a jff input as a Multi Tape Non-Deterministic Turing Machine.

          :param: jff input;
          :return: *(dict)* representing a Multi Tape Turing Machine.
          """
          xmlFile = xmltodict.parse(input_jff)
          
          Q = set()
          Sigma = set()
          Gamma = set()
          Gamma.add(S_BLK)
          delta = {}
          F = set()
          q0 = None

          states_aux = {}

          for s in xmlFile['structure']['automaton']['state']:
            states_aux[s["@id"]] = s["@name"]
            Q.add(s["@name"])
            if 'initial' in s:
              q0 = s["@name"]
            if 'final' in s:
              F.add(s["@name"])
          if 'tapes' in xmlFile['structure']:
            nTapes = int(xmlFile['structure']['tapes'])
          else:
            nTapes = 1
          for t in xmlFile['structure']['automaton']['transition']:
            sFrom = states_aux[t["from"]]
            sTo = states_aux[t["to"]]
            listFrom = [sFrom]
            listTo = [sTo] 
          
            for i in range(nTapes):
              sRead = t["read"][i]["#text"] if "#text" in t["read"][i] else S_BLK
              sWrite = t["write"][i]["#text"] if "#text" in t["write"][i] else S_BLK
              listFrom.append(sRead)
              listTo.append(sWrite)
              if sRead != S_BLK: Sigma.add(sRead)
              if sWrite != S_BLK: Sigma.add(sWrite)
              Gamma.add(sRead)
              Gamma.add(sWrite)
            for i in range(nTapes):
              sMove = t["move"][i]["#text"] if "#text" in t["move"][i] else S_BLK
              listTo.append(sMove)
            if tuple(listFrom) in delta: delta[tuple(listFrom)].add(tuple(listTo))
            else: delta[tuple(listFrom)] = {tuple(listTo)}
          return Q, Sigma, Gamma, delta, q0, F

    def __init__(self, Q={}, Sigma={}, Gamma={}, delta={}, q0=0, blank_as=S_BLK,  F=set(), MTs={}, input_jff=None):
        if input_jff != None:          
          Q,Sigma,Gamma,delta,q0,F = MTNDMF.jffToMT(input_jff)

        b, ERROR = MTNDMF.valida(Q,Sigma,Gamma,delta,q0,blank_as,F)
        if not b: 
          print(f"{bcolors.FAIL}Os seguintes erros foram encontrados:\n")
          print(',\n'.join(ERROR))

        self.states = Q
        self.inputAlphabet = Sigma
        self.tapeAlphabet = Gamma
        self.transition = delta
        self.startState = q0
        self.acceptStates = F
        self.traces = []
        self.MTs = MTs
        self.blank_sym = blank_as#self.default['blank']
        self.left_sym = self.default['left']
        self.right_sym = self.default['right']

        self.keep_traces = True
        self.show_steps = False
        self.ntapes = max([len(t)-1 for t in self.transition])
 
        self.iniciar()

 
    def defaults():
        return __class__.default['blank'], __class__.default['right'], __class__.default['left']
 
    def config(self, blank_sym=None, right_sym=None, left_sym=None, keep_traces=None):
        if blank_sym != None: self.blank_sym = blank_sym 
        if left_sym != None: self.left_sym = left_sym
        if right_sym != None: self.right_sym = right_sym
        if keep_traces != None: self.keep_traces = keep_traces
 
  
    def hasNext(self):
        return self.traces and (not self.acceptTraces())
    
    def acceptTraces(self):
        return [t for t in self.traces if t[-1][0] in self.acceptStates and t[-1][3]==self]
        
    def getQ0(self):
      return self.startState

    def getTransition(self):
      return self.transition
      
    def getTraces(self):
      return self.traces

    def getNameMT(self, M):
      for name, MTNDMF in MTs.items():  # retorna o nome da MT
        if MTNDMF == M: return name
      return ''

        # Uma descrição instântanea é formada por:
        #     - estado
        #     - lista de posições (cabeças das fitas)
        #     - lista do conteúdos das fitas 
        #     - Máquina de Turing
        # Inicializa a lista de Trace com um único trace, contendo a ID inicial
        # Uma lista de Traces, na qual cada trace é formada por uma lista de descrições instântaneas (histórico).
    def iniciar(self, input_string_tapes='', head_tapes=None, start_state=None):
        if start_state != None: self.startState = start_state
        if head_tapes==None:
          head_tapes = [0]*(self.ntapes)
        if type(input_string_tapes) is list:
            input_string_tapes = [list(input_string) for input_string in input_string_tapes]
        else: 
          input_list = list(input_string_tapes) if input_string_tapes else [self.blank_sym]
          input_list = input_list + [self.blank_sym]*(1-len(input_list))
          input_string_tapes = [input_list] + [[self.blank_sym]]*(self.ntapes-1)
        self.startId = (self.startState, head_tapes, input_string_tapes, self)
        self.traces = [ [self.startId] ]         

    def aceita(self, input_string_tapes=None, head_tapes=None, start_state=None, max_steps=1000,show_steps=False):
        self.iniciar(input_string_tapes=input_string_tapes, head_tapes=head_tapes, start_state=start_state)
        if (show_steps):
          self.print_snapshot()
        return self.executar(show_steps=show_steps,max_steps=max_steps)

    def executar(self, max_steps=1000,show_steps=False):   
        while (self.hasNext()): 
          if max_steps == 0: 
            return "Timeout"
          self.step()
          max_steps -= 1
          if (show_steps): 
            self.print_snapshot()
        return self.resultado()
 
    def resultado(self):
        return True if self.acceptTraces() else (False if (not self.traces) else None)

    def step(self):
      updated_traces = []
      for trace in self.traces:
        q, head, tape, M = trace[-1]
        if q in self.MTs:
          M = self.MTs[q]
          if (self.keep_traces):
            updated_traces.append(trace+[(M.startState,head,tape,M)])    
          else:
            updated_traces.append([(M.startState,head,tape,M)])        
        else:
          args = (q,) + tuple([tape[i][h] for i,h in enumerate(head)])            
          if (args in M.getTransition()):
            for values in M.getTransition()[args]:
              h = head[:]
              t = [t[:] for t in tape[:]]
              r = values[0]
              b = values[1:self.ntapes+1]
              m = values[self.ntapes+1:]
              for i in range(self.ntapes):
                t[i][h[i]] = b[i]
                h[i] += 1 if m[i] == self.right_sym else -1 if m[i] == self.left_sym else 0
                if h[i] < 0:
                  h[i] = 0
                  t[i]= [self.blank_sym] +t[i]
                h[i] = 0 if h[i] < 0 else h[i]
                t[i] += [self.blank_sym] if h[i] == len(t[i]) else []
              if (self.keep_traces):
                updated_traces.append(trace+[(r,h,t,M)])    
              else:
                updated_traces.append([(r,h,t,M)])            
          else:
            # Se a máquina não é essa e ela pára e a atual máquina tem q executar a sua transição.
            if M!= self:
              stateName  = self.getNameMT(M)
              args = (stateName,) + tuple([tape[i][h] for i,h in enumerate(head)])
              if (args in self.getTransition()):
                for values in self.getTransition()[args]:
                  h = head[:]
                  t = [t[:] for t in tape[:]]
                  r = values[0]
                  b = values[1:self.ntapes+1]
                  m = values[self.ntapes+1:]
                  for i in range(self.ntapes):
                    t[i][h[i]] = b[i]
                    h[i] += 1 if m[i] == self.right_sym else -1 if m[i] == self.left_sym else 0
                    if h[i] < 0:
                      h[i] = 0
                      t[i]= [self.blank_sym] +t[i]
                    h[i] = 0 if h[i] < 0 else h[i]
                    t[i] += [self.blank_sym] if h[i] == len(t[i]) else []
                  if (self.keep_traces):
                    updated_traces.append(trace+[(r,h,t,self)])    
                  else:
                    updated_traces.append([(r,h,t,self)])
      self.traces = updated_traces         

    def display_resultados(self, casos_testes, max_steps=1000):
      df = pd.DataFrame.from_dict(casos_testes,orient='index',
                       columns=['Esperado'])
      lResult = []
      for palavra in casos_testes.keys():
        lResult.append(self.aceita(palavra,max_steps=max_steps))
      df['Resultado'] =lResult
      df.reset_index(inplace=True)
      df.rename(columns={'index':'Palavra'},inplace=True)
      acertos = df['Esperado'][df['Esperado']==df['Resultado']].count()
      casos = len(lResult)
      print('Acertou {:.2f}% ({} de {})'.format(acertos/casos*100, acertos, casos))
      display(df)

    def print_snapshot(self):
      ntraces = 1
      sTraces = []
      for key in self.snapshot().keys():
        if(key[0]>ntraces):
          ntraces = key[0]
      for n in range(ntraces):
        sIds = ''
        if(self.ntapes>1):
          sIds = '['#S_LEFT_ANGLE_BRACKET             
        for i in range(self.ntapes):
          if(i>0): sIds+=', '
          if (n,i) in self.snapshot():
            sIds += self.snapshot()[n,i]
        if(self.ntapes>1): sIds += ']'#S_RIGHT_ANGLE_BRACKET 
        if sIds!='': sTraces.append(sIds) 
      if sTraces != []:
        print(', '.join(sTraces))

    def snapshot_id_tape(self, id, tape_i):
      q, head, tape, M = id
      if M!= self: 
        return f"{self.getNameMT(M)+':'+''.join(tape[tape_i][:head[tape_i]])}{bcolors.OKBLUE}{q}{bcolors.ENDC}{''.join(tape[tape_i][head[tape_i]:])}"
      if self.MTs == {}:
        return f"{''.join(tape[tape_i][:head[tape_i]])}{bcolors.OKBLUE}{q}{bcolors.ENDC}{''.join(tape[tape_i][head[tape_i]:])}"
      else:
        return f"{'self:'+''.join(tape[tape_i][:head[tape_i]])}{bcolors.OKBLUE}{q}{bcolors.ENDC}{''.join(tape[tape_i][head[tape_i]:])}"

    def snapshot(self):
        snap = {}
        for n, trace in enumerate(self.traces):
          for i in range(self.ntapes):
            snap[(n,i)] = self.snapshot_id_tape(trace[-1],i)
        return snap

    def snapshot_all(self, size=10):
      l = []
      for i in range(self.ntapes):
        l.append(self.graphic_snapshot(size=size, tape_i=i))
      return l

    def graphic_snapshot_all(self, size=10):
      l = self.snapshot_all()
      for i in l:
        display(i)

    def display(self,blank_as=S_BLK_BOX,highlight=[],highlightNonDeterministic=False, turing_name = '', tape_size=10):
      display(self.visualizar(blank_as=blank_as,highlight=highlight, highlightNonDeterministic=highlightNonDeterministic, turing_name = turing_name))
      self.graphic_snapshot_all(size=tape_size)

    def graphic_snapshot(self,id=None, size=10, tape_i=0):
        s = Digraph('tape '+str(tape_i+1), filename='tm_snapshot.gv',
                    node_attr={'shape': 'record','width':'0.1'})
        if id==None:
          id = self.traces[0][-1]
        q, head, tape, M = id
        
        if M != self:
          s.node('state', '<f0>{}:{}'.format(self.getNameMT(M),q))
        else:
          s.node('state', '<f0>{}'.format(q))
        h = head[tape_i]
        t = ['...']+ [self.blank_sym]+tape[tape_i] + [self.blank_sym]*(size-1-len(tape[tape_i])) + ['...']
        fields = ['<f{}>{}'.format(i,a) for i,a in enumerate(t)]
        s.node('tape', "|".join(fields))
        s.edges([('state:f0', 'tape:f{}'.format(head[tape_i]+2))])

        return s      

    def visualizar(self,blank_as=S_BLK_BOX,highlight=[],highlightNonDeterministic=False, turing_name = ''):             
        f = Digraph('turing machine '+turing_name, filename='tm.gv')
        f.attr(rankdir='LR', size='8,5')
 
        f.attr('node', shape='point')
        f.node('')
 
        for n in self.states:
          shapeType = 'circle'
          if n in self.acceptStates:
            shapeType = 'doublecircle'
          if n in highlight:
            f.node(str(n),color='gray',style='filled', shape=shapeType)
          else:
            f.node(str(n),shape=shapeType)
 
        f.edge('', str(self.startState))
 
        label = {}        
        for key in self.transition:
          for value in self.transition[key]:    
            q,r = key[0],value[0]
            nlabel = ''
            for i in range(self.ntapes):
              if(i>0): nlabel += '{}'.format(self.default['separador_tape'])
              nlabel += '{}{}{},{}'.format(str(key[i+1]),str(self.default['separador']),str(value[i+1]),str(value[self.ntapes+i+1]))
            nlabel = nlabel if blank_as == None else nlabel.replace(self.blank_sym,blank_as)
            label[q,r] = label[q,r]+'\n'+nlabel if (q,r) in label else nlabel

        nonDeterministic = {}
        if highlightNonDeterministic:
          for key in self.transition:
            if len(self.transition[key])>1:
              for value in self.transition[key]:    
                nonDeterministic[key[0],value[0]] = True

        for (q,r) in label:
          if highlightNonDeterministic and (q,r) in nonDeterministic:
            f.edge(str(q),str(r),label=label[q,r],color="blue")#,fontcolor="blue")      
          else:
            f.edge(str(q),str(r),label=label[q,r])      

        return f

    def inicializar_vizualizacao(self, id=None, blank_as=S_BLK_BOX, tamanho_fita = 10, pausa = 0.8, show_other_MTs= True):
        if id==None: id = self.startId
        q, head, tape, M = id
        # Inicializa os displays
        d_MT = display(self.visualizar(highlight=[],blank_as=blank_as),display_id=True)
        d_MTs = {}
        if(show_other_MTs):
          for mt in self.MTs.items():
            d_MTs[mt[0]] = display(mt[1].visualizar(highlight=[],blank_as=blank_as),display_id=True)
        d_Tapes = []
        for i in range(self.ntapes):
          d_Tapes.append(display(self.graphic_snapshot(id,size=tamanho_fita, tape_i=i),display_id=True))

        if M== self:
          d_MT.update(self.visualizar(highlight=[q],blank_as=blank_as))
        else:
          if(show_other_MTs):
            d_MTs[self.getNameMT(M)].update(M.visualizar(highlight=[q],blank_as=blank_as))
        sleep(pausa)
        return d_MT, d_MTs, d_Tapes

    def visualizar_id(self, id, d_MT, d_MTs, d_Tapes, blank_as=S_BLK_BOX, tamanho_fita = 10, pausa = 0.8, pausa_entre_ids = 0, show_other_MTs= True):
      q, head, tape, M = id

      if(pausa_entre_ids>0):
        if (M==self):
          d_MT.update(self.visualizar(highlight=[],blank_as=blank_as))
        if(show_other_MTs):
          for mt in self.MTs.items():
            d_MTs[mt[0]].update(mt[1].visualizar(highlight=[],blank_as=blank_as))
        sleep(pausa_entre_ids)

      for i in range(self.ntapes):
        d_Tapes[i].update(self.graphic_snapshot(id,size=tamanho_fita, tape_i=i))
      if M== self:
        d_MT.update(self.visualizar(highlight=[q],blank_as=blank_as))
        if(show_other_MTs):
          for mt in self.MTs.items():
            d_MTs[mt[0]].update(mt[1].visualizar(highlight=[],blank_as=blank_as))
      else:
        if(show_other_MTs):
          d_MTs[self.getNameMT(M)].update(M.visualizar(highlight=[q],blank_as=blank_as))
      sleep(pausa)

    def visualizar_computacao(self, input_string_tapes='', head_tapes=None, start_state=None, max_steps=1000, blank_as=S_BLK_BOX,tamanho_fita = 10, pausa = 0.8, show_other_MTs= True):
      self.keep_traces = True
      aceita = self.aceita(input_string_tapes=input_string_tapes,  start_state=start_state, head_tapes=head_tapes, show_steps=False, max_steps=max_steps)

      if type(input_string_tapes) is list:
        input_string = input_string_tapes[0]#''.join(input_string_tapes[0])
      else:
        input_string = input_string_tapes

      if aceita == True:
        print(f"{bcolors.OKBLUE}A palavra {input_string} foi aceita pela seguinte computação:\n")
        # Exibe o trace aceito mais à esquerda
        trace = self.acceptTraces()[0]
        # Inicializa os displays
        d_MT, d_MTs, d_Tapes = None, None, None
        for id in trace:
          if d_MT==None:
            d_MT, d_MTs, d_Tapes = self.inicializar_vizualizacao(id, blank_as=blank_as, tamanho_fita = tamanho_fita, pausa = pausa, show_other_MTs=show_other_MTs)
          else:          
            self.visualizar_id(id, d_MT, d_MTs, d_Tapes, blank_as=blank_as, tamanho_fita = tamanho_fita, pausa = pausa, pausa_entre_ids = 0.02,show_other_MTs=show_other_MTs)
      else:
        if aceita == "Timeout": print(f"{bcolors.FAIL}A palavra {input_string} não foi aceita em {max_steps} passos. Veja um exemplo:")
        else: print(f"{bcolors.FAIL}A palavra {input_string} não foi aceita por nenhuma computação. Veja um exemplo:")
        # Inicializa a MT
        self.iniciar(input_string_tapes=input_string_tapes,  start_state=start_state, head_tapes=head_tapes)
        id_inicial = self.startId#self.traces[0][0]
        # Inicializa os displays
        d_MT, d_MTs, d_Tapes = self.inicializar_vizualizacao(id_inicial, blank_as=blank_as, tamanho_fita = tamanho_fita, show_other_MTs=show_other_MTs)
        # Exibe o trace mais à esquerda
        #Visualiza a Id Inicial
        self.visualizar_id(id_inicial, d_MT, d_MTs, d_Tapes, blank_as=blank_as, tamanho_fita = tamanho_fita, pausa = pausa, show_other_MTs=show_other_MTs)
        # Inicia a computação
        while (self.hasNext()): 
          if max_steps == 0: 
#            print(f"{bcolors.FAIL}A palavra {input_string} não foi aceita em {max_steps} passos.")
            return False
          self.step()
          max_steps -= 1
          if not self.traces: break
          self.traces = [self.traces[0]]
          id = self.traces[0][-1]
          self.visualizar_id(id, d_MT, d_MTs, d_Tapes, blank_as=blank_as, tamanho_fita = tamanho_fita, pausa = pausa, pausa_entre_ids = 0.02, show_other_MTs=show_other_MTs)


    def simular(self, input_string='', max_steps=200, blank_as=S_BLK_BOX,tamanho_fita = 10, pausa = 0.8, show_other_MTs=True):
      layout = widgets.Layout(width='750px')
      w_max_steps = widgets.IntSlider(
          description="Max. Passos.",
          value=max_steps,
          min=10,
          step=10,
          max=1000,
          description_tooltip='Defina o número máximo de passos executados pela máquina de turing.',
          disabled=False,
          continuous_update=False,
          orientation='horizontal',
          readout=True
      )
      speed = widgets.FloatSlider(
          value=0.8,
          min=0,
          max=3.0,
          step=0.1,
          description='Pausa:',
          description_tooltip='Defina a pausa (em segundos) de exibição entre as descrições instantâneas da simulação.',
          disabled=False,
          continuous_update=False,
          orientation='horizontal',
          readout=True,
          readout_format='.1f',
      )
      step = widgets.Button(description="Passo-a-Passo")
      run = widgets.Button(description="Simular")
      reset = widgets.Button(description="Limpar")
      input = widgets.Text(
          value=input_string,
          placeholder='Digite a entrada',
          description='Entrada:',
          disabled=False,
          layout= layout
      )
      output = widgets.Output()
      wButtons = widgets.HBox([step,run,speed,w_max_steps])
      wInputBox= widgets.HBox([input, reset])
      allButtons = widgets.VBox([wInputBox,wButtons])
      display(allButtons,output)
      self.d_MT = None
      self.d_MTs = None
      self.d_Tapes = None

      def on_button_step_clicked(_):
        run.disabled = True
        speed.disabled = True
        input.disabled = True
        step.disabled = True
        w_max_steps.disabled = True
  
        if self.d_MT == None:
          # Inicializa a MT
          self.iniciar(input.value)
          #self.id_inicial = self.startId#self.traces[0][0]
          # Inicializa os displays
          self.d_MT, self.d_MTs, self.d_Tapes = self.inicializar_vizualizacao(self.startId, blank_as=blank_as, tamanho_fita = tamanho_fita, show_other_MTs=show_other_MTs)
          #Visualiza a Id Inicial
          self.visualizar_id(self.startId, self.d_MT, self.d_MTs, self.d_Tapes, blank_as=blank_as, tamanho_fita = tamanho_fita, pausa = pausa, pausa_entre_ids = 0, show_other_MTs=show_other_MTs)
        elif self.hasNext():
          self.step()
          if self.traces:
            self.traces = [self.traces[0]]
            id = self.traces[0][-1]
            self.visualizar_id(id, self.d_MT, self.d_MTs, self.d_Tapes, blank_as=blank_as, tamanho_fita = tamanho_fita, pausa = pausa, pausa_entre_ids = 0, show_other_MTs=show_other_MTs)
            if self.resultado(): 
              with output:
                print(f"{bcolors.OKBLUE}A palavra {input.value} foi aceita.\n")
              step.disabled = True
              return
          else:
            if self.resultado(): 
              with output:
                print(f"A palavra {input.value} foi aceita.\n")
            else:
              with output:
                self.keep_traces = True
                aceita = self.aceita(input.value,show_steps=False)
                if aceita==True: print(f"{bcolors.FAIL}A palavra {input.value} não foi aceita por esta computação.\n{bcolors.OKGREEN}Todavia, há uma computação que reconhece essa palavra. Clique no botão Simular para visualizar uma computação que reconhece essa palavra.")
                elif aceita == "Timeout": print(f"{bcolors.FAIL}A palavra {input_string} não foi aceita em {w_max_steps.value} passos.")
                else: print(f"{bcolors.FAIL}A palavra {input.value} não foi aceita por nenhuma computação.\n")
            step.disabled = True
            return
        else:
          if not self.resultado():
            with output:
              print(f"{bcolors.FAIL}A palavra {input.value} não foi aceita por nenhuma computação.\n")
            step.disabled = True
            return
        step.disabled = False

      step.on_click(on_button_step_clicked)

      def on_button_run_clicked(_):
        reset.disabled = True
        step.disabled = True
        run.disabled = True
        input.disabled = True
        speed.disabled = True
        w_max_steps.disabled = True
        self.visualizar_computacao(input.value, pausa = speed.value,max_steps=w_max_steps.value, show_other_MTs=show_other_MTs)
        reset.disabled = False

      run.on_click(on_button_run_clicked)

      def clear(_):
        speed.value =0.8
        speed.disabled = False
        reset.disabled = False
        step.disabled = False
        run.disabled = False
        input.disabled = False
        w_max_steps.disabled = False
        w_max_steps.value = max_steps
        input.value = input_string
        clear_output()
        display(allButtons,output)
        self.d_MT = None
        self.d_MTs = None
        self.d_Tapes = None
        output.clear_output()

      reset.on_click(clear)



    def simular_set_start_id(self, input_string='', max_steps=200, blank_as=S_BLK_BOX,tamanho_fita = 10, pausa = 0.8, show_other_MTs=True):
      layout = widgets.Layout(width='850px')
      w_max_steps = widgets.IntSlider(
          description="Max. Passos.",
          value=max_steps,
          min=10,
          step=10,
          max=1000,
          description_tooltip='Defina o número máximo de passos executados pela máquina de turing.',
          disabled=False,
          continuous_update=False,
          orientation='horizontal',
          readout=True
      )
      speed = widgets.FloatSlider(
          value=0.8,
          min=0,
          max=3.0,
          step=0.1,
          description='Pausa:',
          description_tooltip='Defina a pausa (em segundos) de exibição entre as descrições instantâneas da simulação.',
          disabled=False,
          continuous_update=False,
          orientation='horizontal',
          readout=True,
          readout_format='.1f',
      )
      step = widgets.Button(description="Passo-a-Passo")
      run = widgets.Button(description="Simular")
      reset = widgets.Button(description="Limpar")
      wInputs_1 = [widgets.Text(placeholder=f'Prefixo da Fita {i+1}') for i in range(self.ntapes)]
      wInputs_2 = [widgets.Text(placeholder=f'Sufixo da Fita {i+1}') for i in range(self.ntapes)]
      if type(input_string) is list:
        wInputs_2 = [widgets.Text(value=input_string[i], placeholder=f'Sufixo da Fita {i+1}') for i in range(self.ntapes)]
      else:
        wInputs_2 = [widgets.Text(placeholder=f'Sufixo da Fita {i+1}') for i in range(self.ntapes)]
        wInputs_2[0].value=input_string
      wStates = widgets.Dropdown(description="Estado:",options=sorted(list(self.states)),value=self.startState,style= {'description_width': '0px'})      
      wText = widgets.HTML(value="Defina a Descrição Instantânea Inicial da Máquina de Turing")
      wAll = widgets.HBox([widgets.VBox(wInputs_1),wStates,widgets.VBox(wInputs_2)])

      output = widgets.Output()
      wButtons = widgets.HBox([reset,step,run,speed,w_max_steps])
      display(wText,wAll,wButtons,output)
      self.d_MT = None
      self.d_MTs = None
      self.d_Tapes = None
      palavra = ''

      def get_input_id():
        state = wStates.value
        lTapes = []
        lPos = []
        for i in range(self.ntapes):
          pre_tape_i = wInputs_1[i].value
          suf_tape_i = wInputs_2[i].value if wInputs_2[i].value else ' '
          tape_i = (pre_tape_i+suf_tape_i)
          lTapes.append(tape_i) 
          pos_i = len(pre_tape_i)
          lPos.append(pos_i)
        return state, lPos, lTapes


      def on_button_step_clicked(_):
        run.disabled = True
        speed.disabled = True
        wStates.disabled = True
        for i in range(self.ntapes):
          wInputs_1[i].disabled = True
          wInputs_2[i].disabled = True
        step.disabled = True
        w_max_steps.disabled = True
        
        start_state, head_tapes, input_string_tapes = get_input_id()
        if input_string_tapes is list:
          palavra = input_string_tapes[0]
        else:
          palavra = input_string_tapes

        if self.d_MT == None:
          # Inicializa a MT
          self.iniciar(start_state=start_state, head_tapes=head_tapes, input_string_tapes=input_string_tapes)
          # Inicializa os displays
          self.d_MT, self.d_MTs, self.d_Tapes = self.inicializar_vizualizacao(self.startId, blank_as=blank_as, tamanho_fita = tamanho_fita, show_other_MTs=show_other_MTs)
          #Visualiza a Id Inicial
          self.visualizar_id(self.startId, self.d_MT, self.d_MTs, self.d_Tapes, blank_as=blank_as, tamanho_fita = tamanho_fita, pausa = pausa, pausa_entre_ids = 0, show_other_MTs=show_other_MTs)
        elif self.hasNext():
          self.step()
          if self.traces:
            self.traces = [self.traces[0]]
            id = self.traces[0][-1]
            self.visualizar_id(id, self.d_MT, self.d_MTs, self.d_Tapes, blank_as=blank_as, tamanho_fita = tamanho_fita, pausa = pausa, pausa_entre_ids = 0, show_other_MTs=show_other_MTs)
            if self.resultado(): 
              with output:
                print(f"{bcolors.OKBLUE}A palavra {palavra} foi aceita.\n")
              step.disabled = True
              return
          else:
            if self.resultado(): 
              with output:
                print(f"A palavra {palavra} foi aceita.\n")
            else:
              with output:
                self.keep_traces = True
                aceita = self.aceita(start_state=start_state, head_tapes=head_tapes, input_string_tapes=input_string_tapes,show_steps=False)
                if aceita==True: print(f"{bcolors.FAIL}A palavra {palavra} não foi aceita por esta computação.\n{bcolors.OKGREEN}Todavia, há uma computação que reconhece essa palavra. Clique no botão Simular para visualizar uma computação que reconhece essa palavra.")
                elif aceita == "Timeout": print(f"{bcolors.FAIL}A palavra {palavra} não foi aceita em {w_max_steps.value} passos.")
                else: print(f"{bcolors.FAIL}A palavra {palavra} não foi aceita por nenhuma computação.\n")
            step.disabled = True
            return
        else:
          if not self.resultado():
            with output:
              print(f"{bcolors.FAIL}A palavra {palavra} não foi aceita por nenhuma computação.\n")
            step.disabled = True
            return
        step.disabled = False

      step.on_click(on_button_step_clicked)

      def on_button_run_clicked(_):
        reset.disabled = True
        step.disabled = True
        run.disabled = True
        wStates.disabled = True
        for i in range(self.ntapes):
          wInputs_1[i].disabled = True
          wInputs_2[i].disabled = True
        speed.disabled = True
        w_max_steps.disabled = True
        start_state, head_tapes, input_string_tapes = get_input_id()
        self.visualizar_computacao(start_state=start_state, head_tapes=head_tapes, input_string_tapes=input_string_tapes, pausa = speed.value,max_steps=w_max_steps.value, show_other_MTs=show_other_MTs)
        reset.disabled = False

      run.on_click(on_button_run_clicked)

      def clear(_):
        speed.value =0.8
        speed.disabled = False
        reset.disabled = False
        step.disabled = False
        wStates.disabled = False
        for i in range(self.ntapes):
          wInputs_1[i].disabled = False
          wInputs_2[i].disabled = False
        run.disabled = False
        w_max_steps.disabled = False
        w_max_steps.value = max_steps
        clear_output()
        display(wText,wAll,wButtons,output)
        self.d_MT = None
        self.d_MTs = None
        self.d_Tapes = None
        output.clear_output()

      reset.on_click(clear)








class MT(MTNDMF):

    @staticmethod
    def jffToMT(input_jff):
          """ Convert a jff input as a Turing Machine.

          :param: jff input;
          :return: *(dict)* representing a Turing Machine.
          """
          xmlFile = xmltodict.parse(input_jff)
          
          Q = set()
          Sigma = set()
          Gamma = set()
          Gamma.add(S_BLK)
          delta = {}
          F = set()
          q0 = None

          states_aux = {}
          for s in xmlFile['structure']['automaton']['state']:
            states_aux[s["@id"]] = s["@name"]
            Q.add(s["@name"])
            if 'initial' in s:
              q0 = s["@name"]
            if 'final' in s:
              F.add(s["@name"])
          for t in xmlFile['structure']['automaton']['transition']:
            sFrom = states_aux[t["from"]]
            sTo = states_aux[t["to"]]
            sRead = t["read"] if t["read"]!=None else S_BLK
            sWrite = t["write"] if t["write"]!=None else S_BLK
            sMove = t["move"]
            delta[sFrom,sRead] = (sTo, sWrite,sMove)
            if sRead != S_BLK: Sigma.add(sRead)
            if sWrite != S_BLK: Sigma.add(sWrite)
            Gamma.add(sRead)
            Gamma.add(sWrite)
          return Q,Sigma,Gamma,delta,q0,F


    def __init__(self, Q=[], Sigma=[], Gamma=[], delta={}, q0=0, blank_as=S_BLK, F=set(), MTs={},input_jff=None):
        if input_jff != None:          
          Q,Sigma,Gamma,delta,q0,F = MT.jffToMT(input_jff)
        super().__init__(Q,Sigma,Gamma,{x:{delta[x]} for x in delta},q0,blank_as, F,MTs)


class MTMF(MTNDMF):
    @staticmethod
    def jffToMT(input_jff):
          """ Convert a jff input as a Multi Tape Turing Machine.

          :param: jff input;
          :return: *(dict)* representing a Multi Tape Turing Machine.
          """
          xmlFile = xmltodict.parse(input_jff)
          
          Q = set()
          Sigma = set()
          Gamma = set()
          Gamma.add(S_BLK)
          delta = {}
          F = set()
          q0 = None

          states_aux = {}

          for s in xmlFile['structure']['automaton']['state']:
            states_aux[s["@id"]] = s["@name"]
            Q.add(s["@name"])
            if 'initial' in s:
              q0 = s["@name"]
            if 'final' in s:
              F.add(s["@name"])
          if 'tapes' in xmlFile['structure']:
            nTapes = int(xmlFile['structure']['tapes'])
          else:
            nTapes = 1
          for t in xmlFile['structure']['automaton']['transition']:
            sFrom = states_aux[t["from"]]
            sTo = states_aux[t["to"]]
            listFrom = [sFrom]
            listTo = [sTo] 
          
            for i in range(nTapes):
              sRead = t["read"][i]["#text"] if "#text" in t["read"][i] else S_BLK
              sWrite = t["write"][i]["#text"] if "#text" in t["write"][i] else S_BLK
              listFrom.append(sRead)
              listTo.append(sWrite)
              if sRead != S_BLK: Sigma.add(sRead)
              if sWrite != S_BLK: Sigma.add(sWrite)
              Gamma.add(sRead)
              Gamma.add(sWrite)

            for i in range(nTapes):
              sMove = t["move"][i]["#text"] if "#text" in t["move"][i] else ' '
              listTo.append(sMove)
            delta[tuple(listFrom)] = tuple(listTo)

          return Q,Sigma,Gamma,delta,q0,F


    def __init__(self, Q=[], Sigma=[], Gamma=[], delta={}, q0=0,blank_as=S_BLK,  F=set(),input_jff=None):
        if input_jff != None:          
          Q,Sigma,Gamma,delta,q0,F = MTMF.jffToMT(input_jff)

        super().__init__(Q,Sigma,Gamma,{x:{delta[x]} for x in delta},q0,blank_as,F)

class MTND(MTNDMF):

    @staticmethod
    def jffToMT(input_jff):
          """ Convert a jff input as a  Non-Deterministic Turing Machine.

          :param: jff input;
          :return: *(dict)* representing a  Non-Deterministic Turing Machine.
          """
          xmlFile = xmltodict.parse(input_jff)
          
          Q = set()
          Sigma = set()
          Gamma = set()
          Gamma.add(S_BLK)
          delta = {}
          F = set()
          q0 = None

          states_aux = {}
          for s in xmlFile['structure']['automaton']['state']:
            states_aux[s["@id"]] = s["@name"]
            Q.add(s["@name"])
            if 'initial' in s:
              q0 = s["@name"]
            if 'final' in s:
              F.add(s["@name"])
          for t in xmlFile['structure']['automaton']['transition']:
            sFrom = states_aux[t["from"]]
            sTo = states_aux[t["to"]]
            sRead = t["read"] if t["read"]!=None else S_BLK
            sWrite = t["write"] if t["write"]!=None else S_BLK
            sMove = t["move"]
            if (sFrom,sRead) in delta: delta[sFrom,sRead].add((sTo, sWrite,sMove))
            else: delta[sFrom,sRead] = {(sTo, sWrite,sMove)}
            if sRead != S_BLK: Sigma.add(sRead)
            if sWrite != S_BLK: Sigma.add(sWrite)
            Gamma.add(sRead)
            Gamma.add(sWrite)

          return Q,Sigma,Gamma,delta,q0,F


    def __init__(self, Q=[], Sigma=[], Gamma=[], delta={}, q0=0, blank_as=S_BLK, F=set(),MTs={},input_jff=None):
        if input_jff != None:          
          Q,Sigma,Gamma,delta,q0,F = MTND.jffToMT(input_jff)
        super().__init__(Q,Sigma,Gamma,delta,q0,blank_as,F,MTs)




In [ ]:
#@title Implementação em Python de Funções para Indecidibilidade (Encoder)
#@markdown Esta célula contém a implementação em Python das funções para codificar Máquinas de Turing que iremos utilizar nesse curso. 
#@markdown Não é necessário conhecer o código aqui implementado ou mesmo ter um conhecimento profundo da linguagem Python. Basta acompanhar os exemplos e experimentar construir seus próprios modelos.

#@markdown >*Execute esta célula (`ctrl+enter` ou clicando no botão ao lado) para que o ambiente seja carregado com as funções implementadas.*
class Encoder_MT():    
  def __init__(self, MTNDMF,delimiter_color=True):
    self.startState = MTNDMF.startState
    self.states = MTNDMF.states
    self.blank_sym = MTNDMF.blank_sym
    self.tapeAlphabet = MTNDMF.tapeAlphabet
    self.transition = MTNDMF.transition    
    self.ntapes = MTNDMF.ntapes
    self.acceptStates = MTNDMF.acceptStates

    self.delimiter_color = delimiter_color
    self.setDelimiter_1(self.delimiter_color)
    self.setDelimiter_2(self.delimiter_color)

    self.encodeMoves()
    self.encodeStates()
    self.encodeAlphabet()
    self.encodeTransition()

  def setDelimiter_1(self,delimiter_color=True):
    self.delimiter_1 = f'{bcolors.OKBLUE}1{bcolors.ENDC}' if delimiter_color else '1'
    
  def setDelimiter_2(self,delimiter_color=True):
    self.delimiter_2 = f'{bcolors.CYELLOW}11{bcolors.ENDC}' if delimiter_color else '11'

  def encodeMoves(self,L_encode='0',R_encode='00',S_encode='000'):
    self.move_encode = {}
    self.move_encode['L'] = L_encode
    self.move_encode['R'] = R_encode
    self.move_encode['S'] = S_encode

  # Faz a codificação dos estados em 0s. O 0 será sempre o estado inicial.
  def encodeStates(self,begin_with_startState = True):
    self.states_encode = {}
    lStates = list(self.states)
    lStates.sort()
    if begin_with_startState:
      lStates.remove(self.startState)
      s = list(self.acceptStates)[0]
      lStates.remove(s)
      lStates = [self.startState,s]+lStates
    for i in range(len(lStates)):
      self.states_encode[lStates[i]]='0'*(i+1)

  def encodeAlphabet(self,end_with_blank_sym=True):
    # Faz a codificação dos símbolos do alfabeto Gamma em 0s. O símbolo branco será sempre o 0^{size(Gamma)}.
    lTapeAlphabet = list(self.tapeAlphabet)
    lTapeAlphabet.sort()
    if end_with_blank_sym:
      lTapeAlphabet.remove(self.blank_sym)
      lTapeAlphabet = lTapeAlphabet+[self.blank_sym]
    self.tapeAlphabet_encode = {}
    for i in range(len(lTapeAlphabet)):
      self.tapeAlphabet_encode[lTapeAlphabet[i]]='0'*(i+1)

  def encodeTransition(self):
    # Faz a codificação dos símbolos do alfabeto Gamma em 0s. O símbolo branco será sempre o 0^{size(Gamma)}.
    self.transition_encode = []
    for t in self.transition.keys():
      s_t = self.states_encode[t[0]]
      for i in range(self.ntapes):
        s_t += self.delimiter_1+self.tapeAlphabet_encode[t[i+1]]
      for dt in self.transition[t]:
        s_t_aux = str(s_t) + self.delimiter_1 + self.states_encode[dt[0]] 
        moves_encode = ''
        for i in range(self.ntapes):
          s_t_aux += self.delimiter_1+ self.tapeAlphabet_encode[dt[i+1]]
          moves_encode += self.delimiter_1+self.move_encode[dt[self.ntapes+i+1]]
        self.transition_encode.append(s_t_aux+moves_encode)

  def isTransition(self,s_transition):
    if self.delimiter_color: s_transition = s_transition.replace('1',self.delimiter_1) 
    return s_transition in self.transition_encode

  def equals(self, encoded_MT):
    l_transitions = encoded_MT.split('11')
    if self.delimiter_color: l_transitions = [x.replace('1',self.delimiter_1) for x in l_transitions]    
    return set(self.transition_encode) == set(l_transitions)

  @staticmethod
  def decode(encoded_MT,blank_sym=S_BLK):
    l_transitions = encoded_MT.split('11')
    l_transitions.sort()
    ntapes =  (len(l_transitions[0].split('1'))-2)//3
    Q = set()
    Sigma = set()
    Gamma = set()
    delta = {}
    F = {'q2'}
    q0 = 'q1'
    max_symbol = -1 # variável usada para encontrar o "maior" símbolo que será substituído pelo branco
    for t in l_transitions:
      l_t = t.split('1')
      l_key = []
      l_tape =[]
      l_move =[]
      q = 'q'+str(len(l_t[0]))
      r = 'q'+str(len(l_t[ntapes+1]))
      Q.add(q)
      Q.add(r)
      l_key.append(q)
      for i in range(ntapes):
        s_tape_i = len(l_t[i+1])-1
        if s_tape_i > max_symbol:
          max_symbol = s_tape_i 
        Gamma.add(str(s_tape_i))
        l_key.append(str(s_tape_i))
        s_tape_write_i = len(l_t[ntapes+i+2])-1
        if s_tape_write_i > max_symbol:
          max_symbol = s_tape_write_i
        Gamma.add(str(s_tape_write_i))        
        l_tape.append(str(s_tape_write_i))
        s_move_i = len(l_t[2*ntapes+i+2])
        s_move_i = 'L' if s_move_i == 1 else 'R' if s_move_i == 2 else 'S'
        l_move.append(s_move_i)
      if tuple(l_key) in delta: 
        delta[tuple(l_key)].add(tuple([r]+l_tape+l_move))
      else: 
        delta[tuple(l_key)] = {tuple([r]+l_tape+l_move)} 
    #Se a máquina não tem o branco 000, adiciona-o.
    if max_symbol < 2:
      Gamma.add(blank_sym)
    else:
      # Troca na função delta o último símbolo pelo branco.
      Gamma.remove(str(max_symbol))
      Sigma = Gamma.copy()
      Gamma.add(blank_sym)
      s_max_symbol = str(max_symbol)
      delta_aux = {}
      for t in delta.keys():
        hasBlank = False
        new_key = t
        if(s_max_symbol in t[1:ntapes+1]):
          l_key = [t[0]]
          for i in range(ntapes):
            if(t[i+1]==s_max_symbol): l_key.append(blank_sym)
            else: l_key.append(t[i+1])
          l_key+= t[1+ntapes:]
          new_key = tuple(l_key)
          hasBlank = True
        new_dt = set()
        for dt in delta[t]:
          if(s_max_symbol in dt[1:ntapes+1]):
            l_dt = [dt[0]]
            for i in range(ntapes):
              if(dt[i+1]==s_max_symbol): l_dt.append(blank_sym)
              else: l_dt.append(dt[i+1])
            l_dt+= dt[ntapes+1:]
            new_dt.add(tuple(l_dt))
            hasBlank = True
          else:
            new_dt.add(dt)
        if hasBlank: delta_aux[new_key] = new_dt
        else: delta_aux[t] = delta[t]
      delta = delta_aux
    return MTNDMF(Q,Sigma,Gamma,delta,q0,blank_sym,F)

In [ ]:
#@title Implementação em Python de Funções para Indecidibilidade (Exercícios)
#@markdown Esta célula contém a implementação em Python dos exercícios que iremos utilizar nesse curso. 
#@markdown Não é necessário conhecer o código aqui implementado ou mesmo ter um conhecimento profundo da linguagem Python. Basta acompanhar os exemplos e experimentar construir seus próprios modelos.

#@markdown >*Execute esta célula (`ctrl+enter` ou clicando no botão ao lado) para que o ambiente seja carregado com as funções implementadas.*
from random import randrange
import pandas as pd



class Exercicio_MT:

  @staticmethod    
  def gerar(num_states=5, num_extra_transition=2):
    q0 = 'q1'
    F = {'q2'}
    lQ = ['q'+str(i+1) for i in range(num_states)]
    Q = set(lQ)
    lQ_aux = lQ.copy()
    lQ_aux.remove('q2')
    blank=S_BLK #=' ' 
    Sigma = {'0','1'}
    Gamma = {'0','1',S_BLK}
    lGamma = list(Gamma)
    lMove = ['R','L','S']
    delta = {}
    delta['q1',lGamma[randrange(0,3)]] = {('q3',lGamma[randrange(0,3)],lMove[randrange(0,3)])}
    delta['q'+str(num_states),lGamma[randrange(0,3)]] = {('q2',lGamma[randrange(0,3)],lMove[randrange(0,3)])}
    for i in range(3,num_states):
      delta['q'+str(i),lGamma[randrange(0,3)]] = {('q'+str(i+1),lGamma[randrange(0,3)],lMove[randrange(0,3)])}
    while num_extra_transition>0:
      s_origem = lQ_aux[randrange(0,num_states-1)]
      s_destino = lQ[randrange(0,num_states)]
      t_origem = lGamma[randrange(0,3)]
      t_destino = lGamma[randrange(0,3)]
      move = lMove[randrange(0,3)]
      if (s_origem,t_origem) in delta:
        if (s_destino,t_destino,move) in delta[s_origem,t_origem]:
          continue
        else:
          delta[s_origem,t_origem].add((s_destino,t_destino,move))
          num_extra_transition = num_extra_transition-1
      else:
        delta[s_origem,t_origem] = {(s_destino,t_destino,move)}
        num_extra_transition= num_extra_transition-1
    return MTNDMF(Q,Sigma,Gamma,delta,q0,blank,F)

  @staticmethod    
  def gerar_tapes(M, size=3):
    tapes = []
    s = M.startState
    for t in M.transition.keys():
      if t[0] == s: 
        tapes = [t[i+1] for i in range(M.ntapes)]
    ta_size = len(M.tapeAlphabet)
    l_alphabet = list(M.tapeAlphabet)
    for i in range(1,size):
      tapes = [tapes[i]+str(l_alphabet[randrange(0,ta_size)]) for i in range(M.ntapes)]
    return tapes

  @staticmethod    
  def gerar_2(num_states=5, num_extra_transition=2):
    q0 = 'q1'
    F = {'q2'}
    lQ = ['q'+str(i+1) for i in range(num_states)]
    Q = set(lQ)
    lQ_aux = lQ.copy()
    lQ_aux.remove('q2')
    blank=S_BLK #=' ' 
    Sigma = {'0','1'}
    Gamma = {'0','1',S_BLK}
    lGamma = list(Gamma)
    lMove = ['R','L','S']
    delta = {}
    delta['q1',lGamma[randrange(0,3)]] = {('q3',lGamma[randrange(0,3)],lMove[0])}
    delta['q'+str(num_states),lGamma[randrange(0,3)]] = {('q2',lGamma[randrange(0,3)],lMove[randrange(0,3)])}
    for i in range(3,num_states):
      delta['q'+str(i),lGamma[randrange(0,3)]] = {('q'+str(i+1),lGamma[randrange(0,3)],lMove[randrange(0,3)])}
    while num_extra_transition>0:
      s_origem = lQ_aux[randrange(0,num_states-1)]
      s_destino = lQ[randrange(0,num_states)]
      t_origem = lGamma[randrange(0,3)]
      t_destino = lGamma[randrange(0,3)]
      move = lMove[randrange(0,3)]
      if (s_origem,t_origem) in delta:
        if (s_destino,t_destino,move) in delta[s_origem,t_origem]:
          continue
        else:
          delta[s_origem,t_origem].add((s_destino,t_destino,move))
          num_extra_transition = num_extra_transition-1
      else:
        delta[s_origem,t_origem] = {(s_destino,t_destino,move)}
        num_extra_transition= num_extra_transition-1
    return MTNDMF(Q,Sigma,Gamma,delta,q0,blank,F)

  @staticmethod    
  def encode(input_string='',num_states=4, num_extra_transition=2):
      layout = widgets.Layout(width='50%')
      run = widgets.Button(description="Verificar")
      input = widgets.Text(
          value=input_string,
          placeholder='Digite o código em binário da MT acima:',
          description='MT (binário):',
          layout=layout
          )
      M = Exercicio_MT.gerar(num_states, num_extra_transition)
      encoder = Encoder_MT(M)
      outputMT = widgets.Output()
      output = widgets.Output()
      
      with outputMT:
        display(M.visualizar())

      display(outputMT, input, run,output)

      def on_button_run_clicked(_):
        output.clear_output()
        with output:
          if (encoder.equals(input.value)):
            print(f"\n{bcolors.OKBLUE}Parabéns, você acertou a códificação!")
          else:
            print(f"\n{bcolors.FAIL}Infelizmente, você errou a códificação! Tente novamente!")
      run.on_click(on_button_run_clicked)

  # Converte uma string em um número
  @staticmethod    
  def T(s):
    s = '1'+s
    z = len(s)
    r = 0
    for x in range (z):
      y = z-x-1
      r+= (2**y)*int(s[x])
    return r

  @staticmethod
  def gerar_string(size=4):
    l= [str(randrange(0,2)) for i in range(size)]
    return ''.join(l)

  @staticmethod    
  def encode_string(size=4):
      layout = widgets.Layout(width='300px')
      run = widgets.Button(description="Verificar")
      s = Exercicio_MT.gerar_string(size)
      input = widgets.Text(
          value='',
          placeholder=f'Digite a codificação de {s}',
          description=f'',
          layout=layout
          )
      #print(f"Codifique a palavra {s} em um número, usando a função T:")
      output = widgets.Output()
      wInputBox= widgets.HBox([input,run])

      display(wInputBox,output)

      def on_button_run_clicked(_):
        output.clear_output()
        with output:
          if (str(Exercicio_MT.T(s))==(input.value)):
            print(f"\n{bcolors.OKBLUE}Parabéns, você acertou a códificação de {s}!")
          else:
            print(f"\n{bcolors.FAIL}Infelizmente, você errou a códificação de {s}! Tente novamente!")
      run.on_click(on_button_run_clicked)

  @staticmethod    
  def gerar_matriz(size=5):
    A = [x for x in range(size)]
    R = set()
    for i in range(size):
      r_size = randrange(0,size+1)
      for j in range(r_size):
        R.add((j,randrange(0,size+1)))
    return set(A), R

  @staticmethod    
  def get_diagonal(A,R):
    l = []
    for i in range(len(A)):
      if not (i,i) in R:
        l.append(str(i))
    return l
  
  @staticmethod    
  def visualiza_matriz(A,R):
    # Visualiza a diagonalização de uma auto-relação. Note que D é diferente de todo R
    l = []
    d ={}
    A = sorted(list(A))
    for a in A:
      la = []
      R_s = ''
      for b in A:
        if (a,b) in R:
          la.append('x')
        else:
          la.append(' ')  
          
      d[a] = la
      l.append(la)

    df = pd.DataFrame.from_dict(d, orient='index',columns=list(A))
    display(df)

  @staticmethod    
  def visualiza_diagonalizacao(A, R):
    # Visualiza a diagonalização de uma auto-relação. Note que D é diferente de todo R
    l = []
    d ={}
    D = []
    D_s = ''
    A = sorted(list(A))
    for a in A:
      la = []
      R_s = ''
      for b in A:
        if (a,b) in R:
          la.append('x')
          if R_s == '': R_s = str(b)
          else: R_s += ', '+str(b)      
          if a==b: 
            D.append(' ') 
        else:
          if a==b: 
            D.append('x') 
            D_s = str(b) if D_s=='' else D_s+', '+str(b)
          la.append(' ')  
          
      la.append('R_'+str(a)+'={'+R_s+'}')
      d[a] = la
      l.append(la)
    D.append('D={'+D_s+'}')
    if 'D' in A: d['D'] = D  
    else: d[' D'] = D

    df = pd.DataFrame.from_dict(d, orient='index',columns=list(A)+['R'])
    display(df)

  @staticmethod    
  def diagonal(size=4):
    layout = widgets.Layout(width='300px')
    run = widgets.Button(description="Verificar")
    input = widgets.Text(
        value='',
        placeholder=f'Digite os elementos da diagonal da matriz',
        description=f'',
        layout=layout
        )
    output = widgets.Output()
    A, R = Exercicio_MT.gerar_matriz(size)
    Exercicio_MT.visualiza_matriz(A,R)
    wInputBox= widgets.HBox([input,run])
    display(wInputBox,output)
    
    def on_button_run_clicked(_):
      output.clear_output()
      with output:
        if (Exercicio_MT.get_diagonal(A,R)== input.value.split(' '))or (Exercicio_MT.get_diagonal(A,R)== [] and input.value.split(' ')==''):
          print(f"\n{bcolors.OKBLUE}Parabéns, você acertou a diagonal!")
          Exercicio_MT.visualiza_diagonalizacao(A, R)
        else:
          print(f"\n{bcolors.FAIL}Infelizmente, você errou a diagonal! Tente novamente!")
    run.on_click(on_button_run_clicked)


In [ ]:
#@title Implementação em Python para Exercícios de PCP
#@markdown Esta célula contém a implementação em Python dos exercícios do Problema de Correspondência de Post (PCP) que iremos utilizar nesse curso. 
#@markdown Não é necessário conhecer o código aqui implementado ou mesmo ter um conhecimento profundo da linguagem Python. Basta acompanhar os exemplos e experimentar construir seus próprios modelos.

#@markdown >*Execute esta célula (`ctrl+enter` ou clicando no botão ao lado) para que o ambiente seja carregado com as funções implementadas.*
import pandas as pd
from IPython.display import HTML
class PCP:
  def __init__(self, pcp):
    self.pcp = pcp

  def to_dataframe(self):
    lI = []
    l0 = []
    l1 =[]
    lC =[]
    i = 1
    for p in self.pcp:
      lI.append('i'+str(i))
      i+=1
      l0.append(p[0])
      l1.append(p[1])
      lC.append(p[1])
    d_pcp = {'indices':lI, 'Lista A':l0, 'Lista B':l1 }
    df = pd.DataFrame.from_dict(d_pcp)
    return df
  
  def display(self):
    display(HTML(self.to_dataframe().to_html(index=False)))

  def display_solution(self, index_solution):
    lA = [self.pcp[s-1][0] for s in index_solution]
    lB = [self.pcp[s-1][1] for s in index_solution]
    lIndices = ['i'+str(i) for i in index_solution]
    data = {'Lista A': lA, 'Lista B': lB}
    df = pd.DataFrame.from_dict(data, orient='index',
                        columns=lIndices)
    display(HTML(df.to_html()))

  def get_solution(self,index_solution):
    w= ''
    x= ''
    for c in index_solution:
      w += self.pcp[c-1][0]
      x += self.pcp[c-1][1]
    return (w,x)

  def is_solution(self,index_solution):
    w, x = self.get_solution(index_solution)
    return w==x
  
  def __eq__(self, other):
    return set(self.cpc)==set(other.cpc)

  def size(self):
    return len(self.pcp)

class PCPM(PCP):
  def __init__(self, pcpm):
    super().__init__(pcpm)

  def is_solution(self,index_solution):
    return super().is_solution(index_solution) and index_solution!= [] and index_solution[0]==1

  def to_PCP(self):
    pcp = []
    w_0 = '*'
    for a in self.pcp[0][0]:
      w_0 += a+'*'
    x_0 = ''
    for b in self.pcp[0][1]:
      x_0 += '*'+b
    pcp.append((w_0,x_0))
    for p in self.pcp:
      new_a = ''
      for a in p[0]:
        new_a += a+'*'
      new_b = ''
      for b in p[1]:
        new_b += '*'+b
      pcp.append((new_a,new_b))
    pcp.append(('$','*$'))
    return pcp

  def solution_PCMP_to_solution_PCP(self, index_solution_pcpm):
    index_solution_pcp = [1]
    for c in index_solution_pcpm[1:]:
      index_solution_pcp.append(c+1)
    index_solution_pcp.append(len(self))
    return index_solution_pcp

class Encoder_LU_to_PCPM:
  def __init__(self, MTNDMF, word, delimiter='#'):
    self.MT = MTNDMF
    self.word = word
    self.delimiter = delimiter

  def encode(self):
    Gamma = self.MT.tapeAlphabet.copy()
    Gamma.remove(self.MT.blank_sym)
    #Appends the initial ID
    l = [(self.delimiter, f'{self.delimiter}{self.MT.startState}{self.word}{self.delimiter}')]
    #Appends the tape symbols and the delimiter
    l += [(x,x) for x in Gamma]+[(self.delimiter,self.delimiter)]
    #Appends the transition correspondences
    for (s_origem,t_origem) in self.MT.transition.keys():
      destino = self.MT.transition[s_origem,t_origem]
      for (s_destino, t_destino, move) in destino: 
        #print('(',s_origem,t_origem,') : (',s_destino, t_destino, move,')')
        if(move=='R'):
          if(t_origem!=self.MT.blank_sym):
            l += [(f'{s_origem}{t_origem}',f'{t_destino}{s_destino}')]
          else:
            l += [(f'{s_origem}{self.delimiter}',f'{t_destino}{s_destino}{self.delimiter}')]
        elif(move=='L'):
          if(t_origem!=self.MT.blank_sym):
            l += [(f'{Z}{s_origem}{t_origem}',f'{s_destino}{Z}{t_destino}') for Z in Gamma]
          else:
            l += [(f'{Z}{s_origem}{self.delimiter}',f'{s_destino}{Z}{t_destino}{self.delimiter}') for Z in Gamma]
        elif(move=='S'):
          if(t_origem!=self.MT.blank_sym):
            l += [(f'{s_origem}{t_origem}',f'{s_destino}{t_destino}')]
          else:
            l += [(f'{s_origem}{self.delimiter}',f'{s_destino}{t_destino}{self.delimiter}')]
    #Appends the correspondece for the accept states
    for s in self.MT.acceptStates:
      for a in Gamma:
        l.append((f'{a}{s}',s))
        for b in Gamma:
          l.append((f'{a}{s}{b}',s))
        l.append((f'{s}{a}',s))
    #Appends the final correspondence
    for s in self.MT.acceptStates:
      l+= [(f'{s}{self.delimiter}{self.delimiter}',f'{self.delimiter}')]
    return PCPM(l)


class Exercicio_PCP:
  @staticmethod    
  def gerar_exemplo(pcp,input_solution=''):
    layout = widgets.Layout(width='300px')
    run = widgets.Button(description="Verificar")
    input = widgets.Text(
        value=input_solution,
        placeholder=f'Digite os indíces de uma solução para o PCP',
        description=f'',
        layout=layout
        )
    output = widgets.Output()
    #pcp = PCP(input_pcp)
    pcp.display()
    wInputBox= widgets.HBox([input,run])
    display(wInputBox,output)
    
    def on_button_run_clicked(_):
      output.clear_output()
      with output:
        solution = [int(i) for i in input.value.strip().split(' ')]
        if (pcp.is_solution(solution)):
          print(f"\n{bcolors.OKBLUE}Parabéns, você encontrou uma solução!")
          pcp.display_solution(solution)
        else:
          print(f"\n{bcolors.FAIL}Infelizmente, você errou a solução! Tente novamente!")
    run.on_click(on_button_run_clicked)

  @staticmethod    
  def converter_pcpm(pcpm, input_solution='', input_pcp=[], input_solution_pcp=''):
    layout = widgets.Layout(width='350px')
    run = widgets.Button(description="Verificar")
    input = widgets.Text(
        value=input_solution,
        placeholder=f'Digite os indíces de uma solução para o PCPM',
        description=f'',
        layout=layout
        )
    input_text_pcp = widgets.Text(
        value=input_solution_pcp,
        placeholder=f'Digite os indíces da solução para o PCP obtida do PCPM',
        description=f'',
        layout=layout
        )    
    k = pcpm.size()
    lI = [widgets.HTML('<center><b>Indices</b></center>')]+[widgets.HTML('<center>i'+str(i+1)+'</center>') for i in range(k+2)]

    if k+2==len(input_pcp):
      lA = [widgets.HTML('<center><b>Lista A</b></center>')]+[widgets.Text(value=input_pcp[i][0].strip(), placeholder=f'Digite w{i}') for i in range(k+2)]
      lB = [widgets.HTML('<center><b>Lista B</b></center>')]+[widgets.Text(value=input_pcp[i][1].strip(), placeholder=f'Digite x{i}') for i in range(k+2)]
    else:
      lA = [widgets.HTML('<center><b>Lista A</b></center>')]+[widgets.Text('', placeholder=f'Digite w{i}') for i in range(k+2)]
      lB = [widgets.HTML('<center><b>Lista B</b></center>')]+[widgets.Text('', placeholder=f'Digite x{i}') for i in range(k+2)]
    wColunas = widgets.HBox([widgets.VBox(lI), widgets.VBox(lA), widgets.VBox(lB)])
    wPCPM = widgets.Output(layout=layout)    
    with wPCPM:
      pcpm.display()
    wAllPCPM = widgets.VBox([widgets.HTML("<b>PCPM</b>"),wPCPM])
    wAllPCP = widgets.VBox([widgets.HTML("<b>PCP</b>"),wColunas])
    output = widgets.Output()
    wPCPM_PCP= widgets.VBox([widgets.HBox([wAllPCPM,wAllPCP]),
                                          widgets.HBox([widgets.HTML("<b>Solução PCPM:</b>",layout=layout), widgets.HTML("<b>Solução PCP:</b>",layout=layout)]), 
                                          widgets.HBox([input,input_text_pcp])])
    display(wPCPM_PCP, run, output)#, layout=layout)
    
    def on_button_run_clicked(_):
      output.clear_output()
      with output:
        solution = [int(i) for i in input.value.strip().split(' ')] if input.value.strip()!= '' else []
        new_solution_pcp = [int(i) for i in input_text_pcp.value.strip().split(' ')] if input_text_pcp.value.strip()!= '' else []
        l_pcp = [(lA[i+1].value.strip(), lB[i+1].value.strip()) for i in range(k+2)] 
        new_pcp = PCP(l_pcp)
        if (set(l_pcp)==set(pcpm.to_PCP())):
          print(f"\n{bcolors.OKBLUE}Parabéns, você converteu corretamente a PCPM em uma PCP!")
          new_pcp.display()
        else:
          print(f"\n{bcolors.FAIL}Infelizmente, você errou a conversão! Tente novamente!")
        if (pcpm.is_solution(solution)):
          print(f"\n{bcolors.OKBLUE}Parabéns, você encontrou uma solução para a PCPM!")
          pcpm.display_solution(solution)
          if (new_pcp.is_solution(new_solution_pcp)):
            print(f"\n{bcolors.OKBLUE}Parabéns, você encontrou uma solução para o PCP!")
            new_pcp.display_solution(solution)
          else:
            print(f"\n{bcolors.FAIL}Infelizmente, você errou a solução da PCP! Tente novamente!")
            new_pcp.display_solution(solution)
        else:
          print(f"\n{bcolors.FAIL}Infelizmente, você errou a solução da PCPM! Tente novamente!")
          pcpm.display_solution(solution)

    run.on_click(on_button_run_clicked)



  @staticmethod    
  def converter_MT(MT_pcpm, word, input_solution=''):
    layout = widgets.Layout(width='350px')
    run = widgets.Button(description="Verificar")
    input_solution = widgets.Text(
        value=input_solution,
        placeholder=f'Digite os indíces de uma solução para o PCPM',
        description=f'',
        layout=layout
        )
    wPCPM_MT = widgets.Output(layout=layout)    
    with wPCPM_MT:
      print("Seja a Máquina de Turing:")
      display(MT_pcpm.visualizar())
      encoded_pcpm = Encoder_LU_to_PCPM(MT_pcpm,word).encode()
      print(f"Seja o PCPM convertido a partir da MT acima e da palavra {word}")
      encoded_pcpm.display()

    output = widgets.Output()
    display(wPCPM_MT, input_solution, run, output)#, layout=layout)
    
    def on_button_run_clicked(_):
      output.clear_output()
      with output:
        solution = [int(i) for i in input_solution.value.strip().split(' ')]
        if (encoded_pcpm.is_solution(solution)):
          print(f"\n{bcolors.OKBLUE}Parabéns, você acertou uma solução que simula a palavra reconhecida pela Máquina de Turing")
        else:
          print(f"\n{bcolors.FAIL}Infelizmente, você ainda não conseguiu encontrar uma solução que simula a palavra reconhecida pela Máquina de Turing. Tente novamente!!")
        encoded_pcpm.display_solution(solution)
    run.on_click(on_button_run_clicked)




In [ ]:
#@title Questão 1 (0,5 pontos): Diagonal de Cantor
#@markdown Execute essa célula para gerar o seu exercício. Então, digite a diagonal, ou seja, o inverso dos elementos da diagonal. 
#@markdown **Observação: os elementos deverão ser separados por espaço.** 
print('Seja a matriz abaixo')
Exercicio_MT.diagonal(size=4)

Seja a matriz abaixo


,0,1,2,3
0,,,x,x
1,x,,,
2,x,,,
3,,,x,


Output()

In [ ]:
#@title Questão 2 (0,5 Pontos): Convertendo Palavras em Binário em Número
#@markdown Execute essa célula para gerar o seu exercício. Então, codifique a Palavra abaixo em um número.. 

Exercicio_MT.encode_string(size=3)

Output()

In [ ]:
#@title Questáo 3 (1,0 Ponto): Codifição de Máquina de Turing em binário
#@markdown Execute essa célula para gerar o seu exercício. Então, codifique a Máquina de Turing abaixo em binário. 

Exercicio_MT.encode(num_states=3, num_extra_transition=2)

Output()

Text(value='', description='MT (binário):', layout=Layout(width='50%'), placeholder='Digite o código em binári…

Button(description='Verificar', style=ButtonStyle())

Output()

In [ ]:
#@title Questáo 3 (2 Pontos): Simule uma Máquina de Turing na Máquina de Turing Universal
#@markdown Execute essa célula para gerar o seu exercício. Então, codifique a Máquina de Turing abaixo em binário. 

M = Exercicio_MT.gerar_2(num_states=3, num_extra_transition=0)
input_strings = Exercicio_MT.gerar_tapes(M)
M.iniciar(input_string_tapes=input_strings)
encoder = Encoder_MT(M,delimiter_color=False)
input_MT_word = input_strings[0]

wOutputMT1 = widgets.Output()
wOutputMTU1 = widgets.Output()
wOutputMT2 = widgets.Output()
wOutputMTU2 = widgets.Output()

wInputs_1 = [widgets.Text(placeholder=f'Prefixo da Fita {i+1}') for i in range(3)]
wInputs_2 = [widgets.Text(placeholder=f'Sufixo da Fita {i+1}') for i in range(3)]
wStates = widgets.Dropdown(description="Estado:",options=sorted(list(['Controle'])),value='Controle', disabled= True,style= {'description_width': '0px'})      
wText = widgets.HTML(value="<h3>Defina a Descrição Instantânea Inicial da Máquina de Turing Universal de M</h3>")
output1 = widgets.Output()



with wOutputMT1:
  display(widgets.HTML(value="<h3>Considere a descrição instântanea inicial da MT M</h3>"))
  M.display()
with wOutputMTU1:
    wAll = widgets.HBox([widgets.VBox(wInputs_1),wStates,widgets.VBox(wInputs_2)])
    display(wText,wAll,output1)


wInputs_1_B = [widgets.Text(placeholder=f'Prefixo da Fita {i+1}') for i in range(3)]
wInputs_2_B = [widgets.Text(placeholder=f'Sufixo da Fita {i+1}') for i in range(3)]
wStates_B = widgets.Dropdown(description="Estado:",options=sorted(list(['Controle'])),value='Controle', disabled= True,style= {'description_width': '0px'})      
wText_B = widgets.HTML(value="<h3>Defina a Descrição Instantânea da Máquina de Turing Universal de M ao lado</h3>")
output2 = widgets.Output()


with wOutputMT2:
  display(widgets.HTML(value="<h3>Considere a computação em um passo da MT M:</h3>"))
  M.step()
  M.display()

with wOutputMTU2:
    wAll_B = widgets.HBox([widgets.VBox(wInputs_1_B),wStates,widgets.VBox(wInputs_2_B)])
    display(wText_B, wAll_B)


wOutput1 = widgets.HBox([wOutputMT1, wOutputMTU1])
wOutput2 = widgets.HBox([wOutputMT2, wOutputMTU2])
display(wOutput1, wOutput2)

In [ ]:
#@title Questão 5 (2,5 pontos): Converter o PCPM em PCP
#@markdown Execute essa célula para gerar o seu exercício. Então, digite uma solução para o PCPM. 
#@markdown **Observação: os índices (i=1,2,3,...) deverão ser separados por espaço.** 
display(widgets.HTML('<h3>Converta a Instância do Problema de Correspondência de Post Modificado (PCPM) abaixo na sua PCP e encontre as soluções da PCPM e PCP.</h3>'))
input_pcpm_1 = [('A','AB'), ('B','CA'), ('CA','A'),('ABC','C')]
input_pcpm_2 = [('A','AB'), ('CA','A'),('B','CA'), ('ABC','C')]
input_pcpm_3 = [('A','AB'), ('ABC','C'), ('B','CA'), ('CA','A')]
input_pcpm_4 = [('A','AB'), ('B','CA'), ('ABC','C'), ('CA','A')]
input_pcpm_5 = [('A','AB'), ('BC', 'CA'), ('AB', 'C'),('CA','A')]
input_pcpm_6 = [('A','AB'), ('AB', 'C'),('BC', 'CA'), ('CA','A')]
input_pcpm_7 = [('A','AB'), ('CA','A'), ('BC', 'CA'), ('AB', 'C')]
input_pcpm_8 = [('A','AB'), ('BC', 'CA'),('CA','A'), ('AB', 'C')]
input_pcpm_n = [input_pcpm_1, input_pcpm_2, input_pcpm_3, input_pcpm_4, input_pcpm_5, input_pcpm_6, input_pcpm_7, input_pcpm_8 ]
n = randrange(0,9)
pcp = input_pcpm_n[n]
input_solution = ''
input_pcp = ''
input_solution_pcp = ''
Exercicio_PCP.converter_pcpm(PCPM(pcp), input_solution=input_solution, input_pcp=input_pcp, input_solution_pcp =input_solution_pcp)


HTML(value='<h3>Converta a Instância do Problema de Correspondência de Post Modificado (PCPM) abaixo na sua PC…

Button(description='Verificar', style=ButtonStyle())

Output()

In [ ]:
#@title Exercício 6 (2,5 pontos): Converter LU em PCPM
#@markdown Execute essa célula para gerar o seu exercício. Então, digite uma solução para o PCPM que simule a computação da palavra na MT. 
#@markdown **Observação: os índices (i=1,2,3,...) deverão ser separados por espaço.** 
Q = {'q1','q2','q3'}
Sigma = {'0','1'}
Gamma = {'0','1',S_BLK_BOX}# ' '
blank=S_BLK_BOX #='☐' 
delta = {('q1','0'):('q2','1','R'),
         ('q1','1'):('q2','0','L'),
         ('q2','1'):('q1','0','R'),
         ('q2',S_BLK_BOX):('q2','1','L'),
         ('q2','0'):('q3','0','L')}
q0 = 'q1'
F ={'q3'}

MT_pcpm = MT(Q,Sigma,Gamma,delta,q0,blank,F)
word = '00'
solution = ''
Exercicio_PCP.converter_MT(MT_pcpm,word,input_solution=solution)

Output(layout=Layout(width='350px'))

Text(value='', layout=Layout(width='350px'), placeholder='Digite os indíces de uma solução para o PCPM')

Button(description='Verificar', style=ButtonStyle())

Output()